# template-project demo

The purpose of this notebook is to demonstrate the functionality of `template-project`.

The demo is organised to show

- Step 1: Explanation

- Step 2: Explanation

Note that when you submit a pull request, you should `clear all outputs` from your python notebook for a cleaner merge.


In [1]:
from pathlib import Path

from oceanarray import readers



In [2]:
import re
import yaml
from pathlib import Path

def parse_rodb_keys_file(filepath):
    """
    Parse a rodb_keys.txt file with MATLAB-style lines into structured dicts.

    Returns a dictionary with a list of entries under the 'RODB_KEYS' key.
    """
    entries = []
    pattern = re.compile(
        r"^\s*'(?P<line>[^']+)';\s*\.\.\.\s*%?\s*(?P<comment>.*)?"
    )

    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if not line.startswith("'"):
                continue

            match = pattern.match(line)
            if not match:
                continue
            # Extract the line content and optional comment
            print(line)
            raw_fields = match.group("line").split()
            if len(raw_fields) < 4:
                continue  # not enough fields

            key = raw_fields[0]
            fmt = raw_fields[1]
            count = int(raw_fields[2])
            code = int(raw_fields[3])
            comment = match.group("comment").strip() if match.group("comment") else None

            entry = {
                "key": key,
                "format": fmt,
                "count": count,
                "code": code,
            }
            if comment:
                entry["comment"] = comment

            entries.append(entry)

    return {"RODB_KEYS": entries}

infile = Path("../oceanarray/config/rodb_keys.txt")  # Replace with your actual file path
rodb_keys_data = parse_rodb_keys_file(infile)

with open("../oceanarray/config/rodb_keys.yaml", "w") as outfile:
    yaml.dump(rodb_keys_data, outfile, sort_keys=False)

print(f"Wrote {len(rodb_keys_data['RODB_KEYS'])} entries to {outfile}")





'Filename            %s       0   -1'; ...  % (raw data) file name
'Shipname            %s       0   -2'; ...  % ship name
'Cruise              %s       0   -3'; ...  % cruise identifier
'Mooring             %s       0   -4'; ...  % mooring identifer
'Model               %s       0   -5'; ...  % model identifer
'Instrument          %s       0  -10'; ...  % instrument identifier
'InstrType           %s       0  -10'; ...  % instrument identifier
'Serial_Number       %d       1  -11'; ...  % serial number
'SerialNumber        %d       1  -11'; ...  % serial number
'Station             %s       0  -12'; ...  % station number or name
'Profile             %s       0  -13'; ...  % profile identifier
'ArgonautSN          %d       1  -15'; ...  % Argonaut Serial number
'MicrocatSN          %d       1  -16'; ...  % MicroCat Serial number
'CTD_File            %s       0  -20'; ...  % CTD file name
'Meteo_File          %s       0  -21'; ...  % meterological data file name
'WaterTemp           %g 

In [3]:
from oceanarray.rodb import rodbload, rodbsave
from pathlib import Path
import tempfile

# Load a short test input (adjust path if needed)
infile = Path("../data/wb1_12_2015_6123_head10.use")
ds = rodbload(infile)

# Save using updated rodbsave
with tempfile.NamedTemporaryFile("w+", suffix=".use", delete=False) as tmp:
    outpath = Path(tmp.name)
    rodbsave(outpath, ds)

# Print results
print(f"Written file: {outpath}\n")
with open(outpath) as f:
    for i, line in enumerate(f):
        print(line.rstrip())
        if i > 20:
            break


Written file: /tmp/tmp8wixlc_f.use

Mooring                = wb1_12_2015
SerialNumber           = 6123
WaterDepth             = 1401
InstrDepth             = 800
Start_Date             = 2015/11/30
Start_Time             = 19:00
End_Date               = 2017/03/28
End_Time               = 14:00
Latitude               = 26 29.920 N
Longitude              = 076 48.850 W
Columns                = YY:MM:DD:HH:T:C:P
2015   11   30   19.00028    7.8594   36.5649   824.0
2015   11   30   20.00028    7.8540   36.5601   826.1
2015   11   30   21.00028    7.8465   36.5519   827.3
2015   11   30   22.00028    7.8213   36.5280   827.9
2015   11   30   23.00028    7.7556   36.4649   828.9
2015   12    1    0.00028    7.6805   36.3907   830.4
2015   12    1    1.00028    7.6567   36.3641   831.3
2015   12    1    2.00028    7.8435   36.5786   831.3
2015   12    1    3.00028    7.9788   36.6738   829.9
2015   12    1    4.00028    8.0262   36.7642   829.2
